In [4]:
import time
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set random seed for reproducibility
np.random.seed(509)

In [9]:
# Ackley Function
def ackley_2d(x: np.ndarray) -> float:
    """
    Ackley Function (2D)
    Global minimum at (0, 0), f(x*) = 0
    Highly multimodal with many local minima
    """
    x1, x2 = x[0], x[1]
    term1 = -20 * np.exp(-0.2 * np.sqrt(0.5 * (x1**2 + x2**2)))
    term2 = -np.exp(0.5 * (np.cos(2 * np.pi * x1) + np.cos(2 * np.pi * x2)))
    return term1 + term2 + 20 + np.e

def ackley_gradient(x: np.ndarray) -> np.ndarray:
    """Gradient of Ackley function"""
    x1, x2 = x[0], x[1]
    
    # Partial derivative w.r.t. x1
    r = np.sqrt(0.5 * (x1**2 + x2**2))
    df_dx1_term1 = 20 * 0.2 * np.exp(-0.2 * r) * (0.5 * 2 * x1) / (2 * r) if r > 1e-10 else 0
    df_dx1_term2 = np.exp(0.5 * (np.cos(2 * np.pi * x1) + np.cos(2 * np.pi * x2))) * \
                   0.5 * (-np.sin(2 * np.pi * x1)) * 2 * np.pi
    df_dx1 = df_dx1_term1 + df_dx1_term2
    
    # Partial derivative w.r.t. x2
    df_dx2_term1 = 20 * 0.2 * np.exp(-0.2 * r) * (0.5 * 2 * x2) / (2 * r) if r > 1e-10 else 0
    df_dx2_term2 = np.exp(0.5 * (np.cos(2 * np.pi * x1) + np.cos(2 * np.pi * x2))) * \
                   0.5 * (-np.sin(2 * np.pi * x2)) * 2 * np.pi
    df_dx2 = df_dx2_term1 + df_dx2_term2
    
    return np.array([df_dx1, df_dx2])


# Rastrigin Function
def rastrigin_2d(x: np.ndarray) -> float:
    """
    Rastrigin Function (2D)
    Global minimum at (0, 0), f(x*) = 0
    Highly multimodal with regularly distributed local minima
    """
    A = 10
    return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))

def rastrigin_gradient(x: np.ndarray) -> np.ndarray:
    """Gradient of Rastrigin function"""
    A = 10
    return 2 * x + A * 2 * np.pi * np.sin(2 * np.pi * x)


# Rosenbrock Function
def rosenbrock_2d(x: np.ndarray) -> float:
    """
    Rosenbrock Function (2D) - "Banana function"
    Global minimum at (1, 1), f(x*) = 0
    Non-convex with a narrow curved valley (not multimodal but challenging)
    """
    x1, x2 = x[0], x[1]
    return (1 - x1)**2 + 100 * (x2 - x1**2)**2

def rosenbrock_gradient(x: np.ndarray) -> np.ndarray:
    """Gradient of Rosenbrock function"""
    x1, x2 = x[0], x[1]
    df_dx1 = -2 * (1 - x1) + 100 * 2 * (x2 - x1**2) * (-2 * x1)
    df_dx2 = 100 * 2 * (x2 - x1**2)
    return np.array([df_dx1, df_dx2])


In [14]:
# Create 3D surface plots for all test functions
test_funcs_3d = [
    ('Ackley', ackley_2d, (-5, 5), (-5, 5)),
    ('Rastrigin', rastrigin_2d, (-5.12, 5.12), (-5.12, 5.12)),
    ('Rosenbrock', rosenbrock_2d, (-2, 2), (-1, 3))
]

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Ackley Function', 'Rastrigin Function', 'Rosenbrock Function'],
    specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]],
    horizontal_spacing=0.05
)

for col_idx, (name, func, x_bounds, y_bounds) in enumerate(test_funcs_3d, start=1):
    # Create grid
    x_range = np.linspace(x_bounds[0], x_bounds[1], 100)
    y_range = np.linspace(y_bounds[0], y_bounds[1], 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = func(np.array([X[i, j], Y[i, j]]))
    
    # Add surface
    fig.add_trace(
        go.Surface(
            x=X, y=Y, z=Z,
            colorscale='Viridis',
            showscale=(col_idx == 3),
            name=name
        ),
        row=1, col=col_idx
    )

fig.update_layout(
    title_text="3D Surface Plots of Test Functions",
    height=500,
    showlegend=False
)

# Update axes labels
for i in range(1, 4):
    fig.update_scenes(
        dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='f(x)',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.3))
        ),
        row=1, col=i
    )

fig.show()

In [11]:
def gradient_descent(func, grad_func, x0: np.ndarray, learning_rate: float = 0.01, 
                     max_iter: int = 1000, tol: float = 1e-6, verbose: bool = False):
    """
    Gradient Descent optimization
    
    Parameters:
    -----------
    func : function that takes x and returns scalar
    grad_func : function that takes x and returns gradient vector
    x0 : initial point
    learning_rate : step size
    max_iter : maximum iterations
    tol : convergence tolerance on gradient norm
    verbose : print progress
    
    Returns:
    --------
    dict with keys: x_final, f_final, n_iterations, execution_time, path, losses, gradient_norms, converged
    """
    x = x0.copy()
    path = []
    losses = []
    gradient_norms = []
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Store current state
        path.append(x.copy())
        f_val = func(x)
        losses.append(f_val)
        
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        gradient_norms.append(grad_norm)
        
        # Check convergence
        if grad_norm < tol:
            if verbose:
                print(f"Converged at iteration {i}")
            execution_time = time.time() - start_time
            return {
                'algorithm': 'Gradient Descent',
                'x_final': x,
                'f_final': func(x),
                'n_iterations': len(losses),
                'execution_time': execution_time,
                'path': np.array(path),
                'losses': np.array(losses),
                'gradient_norms': np.array(gradient_norms),
                'converged': True
            }
        
        # Update step
        x = x - learning_rate * grad
        
        if verbose and i % 100 == 0:
            print(f"Iter {i}: f(x) = {f_val:.6f}, ||grad|| = {grad_norm:.6f}")
    
    execution_time = time.time() - start_time
    return {
        'algorithm': 'Gradient Descent',
        'x_final': x,
        'f_final': func(x),
        'n_iterations': len(losses),
        'execution_time': execution_time,
        'path': np.array(path),
        'losses': np.array(losses),
        'gradient_norms': np.array(gradient_norms),
        'converged': False
    }

In [29]:
def gradient_descent_momentum(func, grad_func, x0: np.ndarray, learning_rate: float = 0.01, 
                              beta: float = 0.9, max_iter: int = 1000, tol: float = 1e-6, verbose: bool = False):

    x = x0.copy()
    v = np.zeros_like(x) # np.zeros()
    path = []
    losses = []
    gradient_norms = []
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Store current state
        path.append(x.copy())
        f_val = func(x)
        losses.append(f_val)
        
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        gradient_norms.append(grad_norm)
        
        # Check convergence
        if grad_norm < tol:
            if verbose:
                print(f"Converged at iteration {i}")
            execution_time = time.time() - start_time
            return {
                'algorithm': 'Gradient Descent Momentum',
                'x_final': x,
                'f_final': func(x),
                'n_iterations': len(losses),
                'execution_time': execution_time,
                'path': np.array(path),
                'losses': np.array(losses),
                'gradient_norms': np.array(gradient_norms),
                'converged': True
            }
        
        # Update step
        v = beta * v - learning_rate * grad
        
        x = x + v
        
        if verbose and i % 100 == 0:
            print(f"Iter {i}: f(x) = {f_val:.6f}, ||grad|| = {grad_norm:.6f}")
    
    execution_time = time.time() - start_time
    return {
        'algorithm': 'Gradient Descent Momentum',
        'x_final': x,
        'f_final': func(x),
        'n_iterations': len(losses),
        'execution_time': execution_time,
        'path': np.array(path),
        'losses': np.array(losses),
        'gradient_norms': np.array(gradient_norms),
        'converged': False
    }

In [39]:
def adagrad(func, grad_func, x0: np.ndarray, learning_rate: float = 0.01, 
                              beta: float = 0.9, max_iter: int = 1000, tol: float = 1e-6, verbose: bool = False, eps=1e-6):

    x = x0.copy()
    # v = np.zeros_like(x) # np.zeros()
    G = np.zeros_like(x)
    path = []
    losses = []
    gradient_norms = []
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Store current state
        path.append(x.copy())
        f_val = func(x)
        losses.append(f_val)
        
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        gradient_norms.append(grad_norm)
        
        # Check convergence
        if grad_norm < tol:
            if verbose:
                print(f"Converged at iteration {i}")
            execution_time = time.time() - start_time
            return {
                'algorithm': 'AdaGrad',
                'x_final': x,
                'f_final': func(x),
                'n_iterations': len(losses),
                'execution_time': execution_time,
                'path': np.array(path),
                'losses': np.array(losses),
                'gradient_norms': np.array(gradient_norms),
                'converged': True
            }
        
        # Update step
        G = G + grad**2
        x = x - learning_rate * grad / (eps + np.sqrt(G)) 
        
        if verbose and i % 100 == 0:
            print(f"Iter {i}: f(x) = {f_val:.6f}, ||grad|| = {grad_norm:.6f}")
    
    execution_time = time.time() - start_time
    return {
        'algorithm': 'AdaGrad',
        'x_final': x,
        'f_final': func(x),
        'n_iterations': len(losses),
        'execution_time': execution_time,
        'path': np.array(path),
        'losses': np.array(losses),
        'gradient_norms': np.array(gradient_norms),
        'converged': False
    }

In [47]:
def rmsprop(func, grad_func, x0: np.ndarray, learning_rate: float = 0.01, 
            beta: float = 0.9, gamma = 0.9, max_iter: int = 1000, tol: float = 1e-6, verbose: bool = False, eps=1e-6):
    x = x0.copy()
    # v = np.zeros_like(x) # np.zeros()
    G = np.zeros_like(x)
    path = []
    losses = []
    gradient_norms = []
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Store current state
        path.append(x.copy())
        f_val = func(x)
        losses.append(f_val)
        
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        gradient_norms.append(grad_norm)
        
        # Check convergence
        if grad_norm < tol:
            if verbose:
                print(f"Converged at iteration {i}")
            execution_time = time.time() - start_time
            return {
                'algorithm': 'RMSProp',
                'x_final': x,
                'f_final': func(x),
                'n_iterations': len(losses),
                'execution_time': execution_time,
                'path': np.array(path),
                'losses': np.array(losses),
                'gradient_norms': np.array(gradient_norms),
                'converged': True
            }
        
        # Update step
        G = gamma * G + (1 - gamma) * grad**2
        x = x - learning_rate * grad / (eps + np.sqrt(G)) 
        
        if verbose and i % 100 == 0:
            print(f"Iter {i}: f(x) = {f_val:.6f}, ||grad|| = {grad_norm:.6f}")
    
    execution_time = time.time() - start_time
    return {
        'algorithm': 'AdaGrad',
        'x_final': x,
        'f_final': func(x),
        'n_iterations': len(losses),
        'execution_time': execution_time,
        'path': np.array(path),
        'losses': np.array(losses),
        'gradient_norms': np.array(gradient_norms),
        'converged': False
    }

In [78]:
def adam(func, grad_func, x0: np.ndarray, learning_rate: float = 0.01, 
            beta1: float = 0.9, beta2 = 0.999, max_iter: int = 1000, tol: float = 1e-6, verbose: bool = False, eps=1e-6):
    x = x0.copy()
    v = np.zeros_like(x) # np.zeros()
    G = np.zeros_like(x)
    path = []
    losses = []
    gradient_norms = []
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Store current state
        path.append(x.copy())
        f_val = func(x)
        losses.append(f_val)
        
        grad = grad_func(x)
        grad_norm = np.linalg.norm(grad)
        gradient_norms.append(grad_norm)
        
        # Check convergence
        if grad_norm < tol:
            if verbose:
                print(f"Converged at iteration {i}")
            execution_time = time.time() - start_time
            return {
                'algorithm': 'ADAM',
                'x_final': x,
                'f_final': func(x),
                'n_iterations': len(losses),
                'execution_time': execution_time,
                'path': np.array(path),
                'losses': np.array(losses),
                'gradient_norms': np.array(gradient_norms),
                'converged': True
            }
        
        # Update step 0.9
        v = beta1 * v + (1 - beta1) * grad
        G = beta2 * G + (1 - beta2) * grad**2
        
        v_hat = v / (1 - beta1**(i+1)) 
        G_hat = G / (1 - beta2**(i+1))
        
        x = x - learning_rate * v_hat / (eps + np.sqrt(G_hat)) 
        
        if verbose and i % 100 == 0:
            print(f"Iter {i}: f(x) = {f_val:.6f}, ||grad|| = {grad_norm:.6f}")
    
    execution_time = time.time() - start_time
    return {
        'algorithm': 'ADAM',
        'x_final': x,
        'f_final': func(x),
        'n_iterations': len(losses),
        'execution_time': execution_time,
        'path': np.array(path),
        'losses': np.array(losses),
        'gradient_norms': np.array(gradient_norms),
        'converged': False
    }

In [79]:
x0 = np.array([1,1])
adam(ackley_2d, ackley_gradient, x0, verbose=True)

Iter 0: f(x) = 3.625385, ||grad|| = 2.315720
Iter 100: f(x) = 3.388344, ||grad|| = 0.273376
Iter 200: f(x) = 3.296298, ||grad|| = 0.001335
Converged at iteration 288


{'algorithm': 'ADAM',
 'x_final': array([0.31878806, 0.31878806]),
 'f_final': 3.2958488668685395,
 'n_iterations': 289,
 'execution_time': 0.02991628646850586,
 'path': array([[1.        , 1.        ],
        [0.99000001, 0.99000001],
        [0.98002533, 0.98002533],
        [0.97005329, 0.97005329],
        [0.96006032, 0.96006032],
        [0.9500263 , 0.9500263 ],
        [0.9399354 , 0.9399354 ],
        [0.92977586, 0.92977586],
        [0.91953961, 0.91953961],
        [0.90922189, 0.90922189],
        [0.8988209 , 0.8988209 ],
        [0.88833762, 0.88833762],
        [0.87777559, 0.87777559],
        [0.86714077, 0.86714077],
        [0.85644135, 0.85644135],
        [0.84568763, 0.84568763],
        [0.83489178, 0.83489178],
        [0.82406766, 0.82406766],
        [0.81323051, 0.81323051],
        [0.80239671, 0.80239671],
        [0.7915834 , 0.7915834 ],
        [0.78080813, 0.78080813],
        [0.77008855, 0.77008855],
        [0.75944198, 0.75944198],
        [0.7488

In [86]:
# Test configurations
test_functions = [
    ('Ackley', ackley_2d, ackley_gradient, np.array([3.0, 3.0])),
    ('Rastrigin', rastrigin_2d, rastrigin_gradient, np.array([4.0, 4.0])),
    ('Rosenbrock', rosenbrock_2d, rosenbrock_gradient, np.array([-1.5, 2.5]))
]

# Algorithm configurations (with special handling for Rosenbrock)
def get_algorithm_params(func_name):
    """Get algorithm parameters based on function type"""
    if func_name == 'Rosenbrock':
        # Rosenbrock needs smaller learning rates due to steep gradients
        return [
            ('GD', gradient_descent, {'learning_rate': 0.001}),
            ('GD+Momentum', gradient_descent_momentum, {'learning_rate': 0.001, 'beta': 0.9}),
            ('AdaGrad', adagrad, {'learning_rate': 0.1}),
            ('RMSProp', rmsprop, {'learning_rate': 0.001, 'gamma': 0.8}),
            ('Adam', adam, {'learning_rate': 0.001, 'beta1': 0.7, 'beta2': 0.7})
        ]
    else:
        # Standard parameters for other functions
        return [
            ('GD', gradient_descent, {'learning_rate': 0.01}),
            ('GD+Momentum', gradient_descent_momentum, {'learning_rate': 0.01, 'beta': 0.9}),
            ('AdaGrad', adagrad, {'learning_rate': 0.5}),
            ('RMSProp', rmsprop, {'learning_rate': 0.01, 'gamma': 0.8}),
            ('Adam', adam, {'learning_rate': 0.01, 'beta1': 0.7, 'beta2': 0.7})
        ]

# Store all results
all_results = {}

# Run all combinations
for func_name, func, grad_func, x0 in test_functions:
    print(f"\n{'='*60}")
    print(f"Testing on {func_name} Function")
    print(f"Initial point: {x0}, f(x0) = {func(x0):.6f}")
    print(f"{'='*60}\n")
    
    all_results[func_name] = {}
    algorithms = get_algorithm_params(func_name)
    
    for alg_name, alg_func, params in algorithms:
        print(f"Running {alg_name}...")
        result = alg_func(func, grad_func, x0, max_iter=1000, tol=1e-6, **params)
        all_results[func_name][alg_name] = result
        
        print(f"  Final x: {result['x_final']}")
        print(f"  Final f(x): {result['f_final']:.6e}")
        print(f"  Iterations: {result['n_iterations']}")
        print(f"  Time: {result['execution_time']:.4f}s")
        print(f"  Converged: {result['converged']}")
        print()


Testing on Ackley Function
Initial point: [3. 3.], f(x0) = 9.023767

Running GD...
  Final x: [2.36257364 2.36257364]
  Final f(x): 9.727481e+00
  Iterations: 134
  Time: 0.0082s
  Converged: True

Running GD+Momentum...
  Final x: [-0.31878804 -0.31878804]
  Final f(x): 3.295849e+00
  Iterations: 289
  Time: 0.0221s
  Converged: True

Running AdaGrad...
  Final x: [2.36257355 2.36257355]
  Final f(x): 9.727480e+00
  Iterations: 25
  Time: 0.0015s
  Converged: True

Running RMSProp...
  Final x: [2.36257358 2.36257358]
  Final f(x): 9.727481e+00
  Iterations: 82
  Time: 0.0065s
  Converged: True

Running Adam...
  Final x: [2.36191777 2.36191777]
  Final f(x): 9.724204e+00
  Iterations: 1000
  Time: 0.1234s
  Converged: False


Testing on Rastrigin Function
Initial point: [4. 4.], f(x0) = 32.000000

Running GD...


  Final x: [3.08612327 3.08612327]
  Final f(x): 2.190575e+01
  Iterations: 1000
  Time: 0.0648s
  Converged: False

Running GD+Momentum...
  Final x: [3.90647598 3.90647598]
  Final f(x): 3.387596e+01
  Iterations: 1000
  Time: 0.0723s
  Converged: False

Running AdaGrad...
  Final x: [2.9848557 2.9848557]
  Final f(x): 1.790920e+01
  Iterations: 90
  Time: 0.0075s
  Converged: True

Running RMSProp...
  Final x: [3.98477387 3.98477387]
  Final f(x): 3.184830e+01
  Iterations: 1000
  Time: 0.0929s
  Converged: False

Running Adam...
  Final x: [3.9784162 3.9784162]
  Final f(x): 3.183922e+01
  Iterations: 1000
  Time: 0.1274s
  Converged: False


Testing on Rosenbrock Function
Initial point: [-1.5  2.5], f(x0) = 12.500000

Running GD...
  Final x: [-0.81205782  0.66752665]
  Final f(x): 3.290096e+00
  Iterations: 1000
  Time: 0.0192s
  Converged: False

Running GD+Momentum...
  Final x: [0.99126658 0.98257431]
  Final f(x): 7.639598e-05
  Iterations: 1000
  Time: 0.0390s
  Converged: 

In [87]:
# Create comparison table
import pandas as pd

for func_name in all_results.keys():
    print(f"\n{func_name} Function Results:")
    print("="*80)
    
    data = []
    for alg_name, result in all_results[func_name].items():
        data.append({
            'Algorithm': alg_name,
            'Final f(x)': f"{result['f_final']:.6e}",
            'Iterations': result['n_iterations'],
            'Time (s)': f"{result['execution_time']:.4f}",
            'Converged': '✓' if result['converged'] else '✗',
            'Final ||grad||': f"{result['gradient_norms'][-1]:.6e}"
        })
    
    df = pd.DataFrame(data)
    print(df.to_string(index=False))
    print()


Ackley Function Results:
  Algorithm   Final f(x)  Iterations Time (s) Converged Final ||grad||
         GD 9.727481e+00         134   0.0082         ✓   9.438423e-07
GD+Momentum 3.295849e+00         289   0.0221         ✓   1.704553e-07
    AdaGrad 9.727480e+00          25   0.0015         ✓   5.688887e-07
    RMSProp 9.727481e+00          82   0.0065         ✓   1.003155e-07
       Adam 9.724204e+00        1000   0.1234         ✗   4.344632e-03


Rastrigin Function Results:
  Algorithm   Final f(x)  Iterations Time (s) Converged Final ||grad||
         GD 2.190575e+01        1000   0.0648         ✗   6.248578e+01
GD+Momentum 3.387596e+01        1000   0.0723         ✗   3.821253e+01
    AdaGrad 1.790920e+01          90   0.0075         ✓   9.205733e-07
    RMSProp 3.184830e+01        1000   0.0929         ✗   2.782719e+00
       Adam 3.183922e+01        1000   0.1274         ✗   4.266639e-01


Rosenbrock Function Results:
  Algorithm   Final f(x)  Iterations Time (s) Converged Final

In [88]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create loss curves for all functions
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Ackley Function', 'Rastrigin Function', 'Rosenbrock Function'],
    x_title='Iteration',
    y_title='Loss (log scale)'
)

colors = ['blue', 'red', 'green', 'purple', 'orange']

for col_idx, func_name in enumerate(['Ackley', 'Rastrigin', 'Rosenbrock'], start=1):
    for color_idx, (alg_name, result) in enumerate(all_results[func_name].items()):
        fig.add_trace(
            go.Scatter(
                x=list(range(len(result['losses']))),
                y=result['losses'],
                mode='lines',
                name=alg_name,
                line=dict(color=colors[color_idx]),
                legendgroup=alg_name,
                showlegend=(col_idx == 1)
            ),
            row=1, col=col_idx
        )

fig.update_yaxes(type="log")
fig.update_layout(
    height=400,
    title_text="Loss vs Iteration (All Algorithms)",
    hovermode='x unified'
)
fig.show()

In [89]:
# Create gradient norm curves
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Ackley Function', 'Rastrigin Function', 'Rosenbrock Function'],
    x_title='Iteration',
    y_title='Gradient Norm (log scale)'
)

for col_idx, func_name in enumerate(['Ackley', 'Rastrigin', 'Rosenbrock'], start=1):
    for color_idx, (alg_name, result) in enumerate(all_results[func_name].items()):
        fig.add_trace(
            go.Scatter(
                x=list(range(len(result['gradient_norms']))),
                y=result['gradient_norms'],
                mode='lines',
                name=alg_name,
                line=dict(color=colors[color_idx]),
                legendgroup=alg_name,
                showlegend=(col_idx == 1)
            ),
            row=1, col=col_idx
        )

fig.update_yaxes(type="log")
fig.update_layout(
    height=400,
    title_text="Gradient Norm vs Iteration (All Algorithms)",
    hovermode='x unified'
)
fig.show()

In [90]:
# Create contour plots with trajectories
for func_name, func, grad_func, x0 in test_functions:
    # Create grid for contour plot
    if func_name == 'Ackley':
        x_range = np.linspace(-5, 5, 200)
        y_range = np.linspace(-5, 5, 200)
        global_min = [0, 0]
    elif func_name == 'Rastrigin':
        x_range = np.linspace(-5.12, 5.12, 200)
        y_range = np.linspace(-5.12, 5.12, 200)
        global_min = [0, 0]
    else:  # Rosenbrock
        x_range = np.linspace(-2, 2, 200)
        y_range = np.linspace(-1, 3, 200)
        global_min = [1, 1]
    
    X, Y = np.meshgrid(x_range, y_range)
    Z = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Z[i, j] = func(np.array([X[i, j], Y[i, j]]))
    
    # Create figure
    fig = go.Figure()
    
    # Add contour plot
    fig.add_trace(go.Contour(
        x=x_range,
        y=y_range,
        z=Z,
        colorscale='Viridis',
        contours=dict(
            start=0,
            end=np.percentile(Z, 90),
            size=(np.percentile(Z, 90) - 0) / 20
        ),
        showscale=False,
        name='Function'
    ))
    
    # Add trajectories for each algorithm
    for color_idx, (alg_name, result) in enumerate(all_results[func_name].items()):
        path = result['path']
        # Subsample path for clarity (plot every 10th point for long trajectories)
        step = max(1, len(path) // 50)
        
        fig.add_trace(go.Scatter(
            x=path[::step, 0],
            y=path[::step, 1],
            mode='lines+markers',
            name=alg_name,
            line=dict(color=colors[color_idx], width=2),
            marker=dict(size=4)
        ))
        
        # Add start and end markers
        fig.add_trace(go.Scatter(
            x=[path[0, 0]],
            y=[path[0, 1]],
            mode='markers',
            marker=dict(size=12, color='white', symbol='circle', 
                       line=dict(color=colors[color_idx], width=2)),
            showlegend=False,
            hovertext=f'{alg_name} start'
        ))
        
        fig.add_trace(go.Scatter(
            x=[path[-1, 0]],
            y=[path[-1, 1]],
            mode='markers',
            marker=dict(size=12, color=colors[color_idx], symbol='star'),
            showlegend=False,
            hovertext=f'{alg_name} end'
        ))
    
    # Add global minimum marker
    fig.add_trace(go.Scatter(
        x=[global_min[0]],
        y=[global_min[1]],
        mode='markers',
        marker=dict(size=15, color='red', symbol='x', line=dict(width=3)),
        name='Global Minimum',
        showlegend=True
    ))
    
    fig.update_layout(
        title=f'Optimization Trajectories on {func_name} Function',
        xaxis_title='x1',
        yaxis_title='x2',
        height=600,
        width=700,
        hovermode='closest'
    )
    
    fig.show()